In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from matplotlib import pyplot as plt
from tqdm import trange
from torch.distributions.normal import Normal
from bnn import BayesBaseModule, BayesConv2d, BayesLinear, BayesModel
from bnn.models import CNN, FCNN
from torch.distributions.laplace import Laplace
from torch.utils.data import random_split
from pathlib import Path

## Dataset

In [2]:
dataset = torchvision.datasets.MNIST('./files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ]))

generator=torch.Generator()
generator.manual_seed(42)
train_set, val_set = random_split(dataset, [50000, 10000], generator=generator)

## Train

In [3]:
distr = Laplace(torch.tensor(0.), torch.tensor(.05))
get_mdl = lambda: (CNN(weight_distribution=distr, bias_distribution=distr), 'cnn')
temperatures = [1, 5e-1, 1e-1, 5e-2, 1e-2, 5e-3, 1e-3]
results = []

run_dir = Path('./runs/')
model_dir = Path('./models/')

for get_mdl in (lambda: (CNN(weight_distribution=distr, bias_distribution=distr), 'cnn'), lambda: (FCNN(weight_distribution=distr, bias_distribution=distr), 'linear')):
    for i, temperature in enumerate(temperatures):
        mdl, name = get_mdl()

        exp_name = f'{name}_laplace_{temperature}'

        model_save_path = model_dir / exp_name
        run_save_path = run_dir / exp_name

        model_save_path.mkdir(exist_ok=True, parents=True)
        run_save_path.mkdir(exist_ok=True, parents=True)

        trainer = BayesModel(train_dataset=train_set,
                        test_dataset=val_set,
                        batch_size=128,
                        architecture=mdl,
                        lr=1e-3,
                        temperature=temperature)

        trainer.fit(n_epochs=100, log_dir=str(run_save_path))
        results.append(trainer.evaluate())

        torch.save(trainer.architecture.state_dict(), model_save_path / 'model.pth')

 23%|██▎       | 23/100 [02:33<08:28,  6.60s/it]

In [ ]:
distr = Laplace(torch.tensor(0.), torch.tensor(.05))
get_mdl = lambda: (CNN(weight_distribution=distr, bias_distribution=distr), 'cnn')
temperatures = [1, 5e-1, 1e-1, 5e-2, 1e-2, 5e-3, 1e-3]
results_long = []

run_dir = Path('./runs/')
model_dir = Path('./models/')

for get_mdl in (lambda: (CNN(weight_distribution=distr, bias_distribution=distr), 'cnn'), lambda: (FCNN(weight_distribution=distr, bias_distribution=distr), 'linear')):
    for i, temperature in enumerate(temperatures):
        mdl, name = get_mdl()

        exp_name = f'long_{name}_laplace_{temperature}'

        model_save_path = model_dir / exp_name
        run_save_path = run_dir / exp_name

        model_save_path.mkdir(exist_ok=True, parents=True)
        run_save_path.mkdir(exist_ok=True, parents=True)

        trainer = BayesModel(train_dataset=train_set,
                        test_dataset=val_set,
                        batch_size=128,
                        architecture=mdl,
                        lr=1e-3,
                        temperature=temperature)

        trainer.fit(n_epochs=1000, log_dir=str(run_save_path))
        results_long.append(trainer.evaluate())

        torch.save(trainer.architecture.state_dict(), model_save_path / 'model.pth')

In [ ]:
import json

with open()